<a href="https://colab.research.google.com/github/putrinahampun/PreparationForTFDSExam/blob/main/ExerciseDicoding/SubmissionA/ProblemA5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# =======================================================================================
# PROBLEM A5
#
# Build and train a neural network model using the Sunspots.csv dataset.
# Use MAE as the metrics of your neural network model.
# We provided code for normalizing the data. Please do not change the code.
# Do not use lambda layers in your model.
#
# The dataset used in this problem is downloaded from kaggle.com/robervalt/sunspots
#
# Desired MAE < 0.15 on the normalized dataset.
# ========================================================================================

import csv
import tensorflow as tf
import numpy as np
import urllib

# DO NOT CHANGE THIS CODE
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)


def solution_A5():
    data_url = 'https://github.com/dicodingacademy/assets/raw/main/Simulation/machine_learning/sunspots.csv'
    urllib.request.urlretrieve(data_url, 'sunspots.csv')

    time_step = []
    sunspots = []

    with open('sunspots.csv') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        next(reader)
        for row in reader:
            sunspots.append(float(row[2]))  # Column index 2 contains sunspot data
            time_step.append(int(row[0]))  # Column index 0 contains time step

    series = np.array(sunspots)  # Convert sunspots to numpy array

    # Normalization Function. DO NOT CHANGE THIS CODE
    min_val = np.min(series)
    max_val = np.max(series)
    series = (series - min_val) / (max_val - min_val)
    time = np.array(time_step)

    # DO NOT CHANGE THIS CODE
    split_time = 3000

    time_train = time[:split_time]
    x_train = series[:split_time]
    time_valid = time[split_time:]
    x_valid = series[split_time:]

    # DO NOT CHANGE THIS CODE
    window_size = 30
    batch_size = 32
    shuffle_buffer_size = 1000

    train_set = windowed_dataset(x_train, window_size=window_size,
                                 batch_size=batch_size, shuffle_buffer=shuffle_buffer_size)

    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv1D(filters=32, kernel_size=5, strides=1, padding="causal", activation="relu",
                               input_shape=[None, 1]),
        tf.keras.layers.LSTM(64, return_sequences=True),
        tf.keras.layers.LSTM(64, return_sequences=True),
        tf.keras.layers.Dense(30, activation="relu"),
        tf.keras.layers.Dense(10, activation="relu"),
        tf.keras.layers.Dense(1)
    ])

    # Compile the model
    model.compile(loss=tf.keras.losses.Huber(),
                  optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                  metrics=["mae"])

    # Train the model
    model.fit(train_set, epochs=10)

    return model

# The code below is to save your model as a .h5 file.
# It will be saved automatically in your Submission folder.
if __name__ == '__main__':
    # DO NOT CHANGE THIS CODE
    model = solution_A5()
    model.save("model_A5.h5")


Epoch 1/10
93/93 [==============================] - 15s 50ms/step - loss: 0.0115 - mae: 0.1070
Epoch 2/10
93/93 [==============================] - 5s 48ms/step - loss: 0.0064 - mae: 0.0797
Epoch 3/10
93/93 [==============================] - 6s 61ms/step - loss: 0.0049 - mae: 0.0695
Epoch 4/10
93/93 [==============================] - 5s 49ms/step - loss: 0.0042 - mae: 0.0644
Epoch 5/10
93/93 [==============================] - 5s 50ms/step - loss: 0.0037 - mae: 0.0607
Epoch 6/10
93/93 [==============================] - 5s 56ms/step - loss: 0.0034 - mae: 0.0583
Epoch 7/10
93/93 [==============================] - 6s 60ms/step - loss: 0.0032 - mae: 0.0564
Epoch 8/10
93/93 [==============================] - 5s 49ms/step - loss: 0.0030 - mae: 0.0550
Epoch 9/10
93/93 [==============================] - 6s 58ms/step - loss: 0.0029 - mae: 0.0538
Epoch 10/10
93/93 [==============================] - 5s 48ms/step - loss: 0.0028 - mae: 0.0530
